# Deep Learning via ANN

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2023-08-12 18:13:07.119881: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
tf.__version__

'2.13.0'

## Importing dataset

In [2]:
dataset = pd.read_csv('AmericanExpress_data.csv')
X = dataset.iloc[:, 0:-1].values
y = dataset.iloc[:, -1].values

In [3]:
print(X)

[[553 'Delhi' 'Female' ... 4 1 274150]
 [447 'Bengaluru' 'Male' ... 4 1 519360]
 [501 'Delhi' 'Female' ... 4 1 545501]
 ...
 [627 'Mumbai' 'Female' ... 4 0 494067]
 [600 'Bengaluru' 'Female' ... 2 1 109375]
 [553 'Delhi' 'Male' ... 4 1 180031]]


## Encoding categorical data

### Gender column : Label Encoding

In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X[:, 2] = label_encoder.fit_transform(X[:, 2])

In [5]:
print(X)

[[553 'Delhi' 0 ... 4 1 274150]
 [447 'Bengaluru' 1 ... 4 1 519360]
 [501 'Delhi' 0 ... 4 1 545501]
 ...
 [627 'Mumbai' 0 ... 4 0 494067]
 [600 'Bengaluru' 0 ... 2 1 109375]
 [553 'Delhi' 1 ... 4 1 180031]]


### Geography column : One hot Encoding

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [7]:
print(X)

[[0.0 1.0 0.0 ... 4 1 274150]
 [1.0 0.0 0.0 ... 4 1 519360]
 [0.0 1.0 0.0 ... 4 1 545501]
 ...
 [0.0 0.0 1.0 ... 4 0 494067]
 [1.0 0.0 0.0 ... 2 1 109375]
 [0.0 1.0 0.0 ... 4 1 180031]]


## Splitting dataset into Training & Test set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## ANN

### Initialization

In [10]:
ann = tf.keras.models.Sequential()

### Adding input layer and first hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding second hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding output layer

In [13]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## ANN Training

### Compiling ANN

In [14]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training on training dataset

In [15]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 120)

Epoch 1/120
249/249 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 0.7832
Epoch 2/120
249/249 [==============================] - 0s 1ms/step - loss: 0.4718 - accuracy: 0.7966
Epoch 3/120
249/249 [==============================] - 0s 986us/step - loss: 0.4510 - accuracy: 0.7956
Epoch 4/120
249/249 [==============================] - 0s 1ms/step - loss: 0.4413 - accuracy: 0.7978
Epoch 5/120
249/249 [==============================] - 0s 1ms/step - loss: 0.4364 - accuracy: 0.8023
Epoch 6/120
249/249 [==============================] - 0s 1ms/step - loss: 0.4335 - accuracy: 0.8082
Epoch 7/120
249/249 [==============================] - 0s 1ms/step - loss: 0.4309 - accuracy: 0.8117
Epoch 8/120
249/249 [==============================] - 0s 995us/step - loss: 0.4281 - accuracy: 0.8159
Epoch 9/120
249/249 [==============================] - 0s 1ms/step - loss: 0.4262 - accuracy: 0.8168
Epoch 10/120
249/249 [==============================] - 0s 970us/step - loss: 0.4237 - 

# Predictions

### Single Prediction

In [19]:
print(ann.predict(sc.transform([[0.0, 1.0, 0.0, 501, 0, 32, 2, 0.0, 4, 1, 545501]])) > 0.5)

1/1 [==============================] - 0s 28ms/step
[[False]]


### Predication on testset

In [17]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 1]
 [0 0]
 [0 0]
 ...
 [0 1]
 [0 0]
 [1 1]]


### Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1523   50]
 [ 241  172]]


0.8534743202416919